In [4]:
import os
import pandas as pd
import psycopg2
import databases.database_utility

conn = psycopg2.connect(
    database="graphyfinancials",
    host="localhost",
    user="derek",
    password="",
    port="5432"
)

ModuleNotFoundError: No module named 'databases'

In [37]:

data_directory = '/Users/derek/Documents/Github_Repos/graphy_financials/data'

tsv_disclosure = []
tsv_coissuer_info = []
tsv_issuer_info = []
tsv_issuer_jurisdiction = []
tsv_issuer_signature = []
tsv_signature = []
tsv_submission = []

# def tsv_processor(arr_tsv: object):
#     for folder_name in 

for folder_name in os.listdir(data_directory):
    folder_path = os.path.join(data_directory, folder_name)
    
    if os.path.isdir(folder_path):
        for root, dirs, files in os.walk(folder_path):
            for file_name in files:
                if file_name.endswith('DISCLOSURE.tsv'):
                    tsv_path = os.path.join(root, file_name)
                    tsv_disclosure.append(tsv_path)
                if file_name.endswith('C_COISSUER_INFORMATION.tsv'):
                    tsv_path = os.path.join(root, file_name)
                    tsv_coissuer_info.append(tsv_path)
                if file_name.endswith('C_ISSUER_INFORMATION.tsv'):
                    tsv_path = os.path.join(root, file_name)
                    tsv_issuer_info.append(tsv_path)
                if file_name.endswith('ISSUER_JURISDICTIONS.tsv'):
                    tsv_path = os.path.join(root, file_name)
                    tsv_issuer_jurisdiction.append(tsv_path)
                if file_name.endswith('ISSUER_SIGNATURE.tsv'):
                    tsv_path = os.path.join(root, file_name)
                    tsv_issuer_signature.append(tsv_path)
                if file_name.endswith('C_SIGNATURE.tsv'):
                    tsv_path = os.path.join(root, file_name)
                    tsv_signature.append(tsv_path)
                if file_name.endswith('SUBMISSION.tsv'):
                    tsv_path = os.path.join(root, file_name)
                    tsv_submission.append(tsv_path)

In [38]:
def tsv_to_dataframe(tsv_list: object, multi_df=False):
    if multi_df == True:
        consolidated_df = []
        for tsv_file in tsv_list:
            df = pd.read_csv(tsv_file, sep='\t')
            consolidated_df.append(df)
        # print(consolidated_df)
        return pd.concat(consolidated_df, ignore_index=True)
    elif multi_df == False:
        df = pd.read_csv(tsv_list)
        return df

disclosure_dataframe = tsv_to_dataframe(tsv_disclosure, multi_df=True)
coissuer_info_dataframe = tsv_to_dataframe(tsv_coissuer_info, multi_df=True)
issuer_info_dataframe = tsv_to_dataframe(tsv_issuer_info, multi_df=True)
issuer_jurisdiction_dataframe = tsv_to_dataframe(tsv_issuer_jurisdiction, multi_df=True)
issuer_signature_dataframe = tsv_to_dataframe(tsv_issuer_signature, multi_df=True)
signature_dataframe = tsv_to_dataframe(tsv_signature, multi_df=True)
submission_dataframes = tsv_to_dataframe(tsv_submission, multi_df=True)

In [39]:
# DISCLOSURES

disclosure_dataframe.rename(columns = {
    'ACCESSION_NUMBER': 'accession_number',
    'COMPENSATIONAMOUNT': 'compensation_amount_description', 
    'FINANCIALINTEREST': 'financial_interest',
    'SECURITYOFFEREDTYPE': 'security_offered_type',
    'SECURITYOFFEREDOTHERDESC': 'security_offered_other_desc',
    'NOOFSECURITYOFFERED': 'no_of_security_offered',
    'PRICE': 'price',
    'PRICEDETERMINATIONMETHOD': 'price_determination_method', 
    'OFFERINGAMOUNT': 'offering_amount',
    'OVERSUBSCRIPTIONACCEPTED': 'oversubscription_accepted',
    'OVERSUBSCRIPTIONALLOCATIONTYPE': 'oversubscription_allocation_type',
    'DESCOVERSUBSCRIPTION': 'desc_oversubscription',
    'MAXIMUMOFFERINGAMOUNT': 'maximum_offering_amount',
    'DEADLINEDATE': 'deadline_date',
    'CURRENTEMPLOYEES': 'current_employees',
    'TOTALASSETMOSTRECENTFISCALYEAR': 'total_assets_most_recent_fiscal_year',
    'TOTALASSETPRIORFISCALYEAR': 'total_assets_prior_fiscal_year',
    'CASHEQUIMOSTRECENTFISCALYEAR': 'cash_equity_most_recent_fiscal_year',
    'CASHEQUIPRIORFISCALYEAR': 'cash_equity_prior_fiscal_year',
    'ACTRECEIVEDRECENTFISCALYEAR': 'act_received_recent_fiscal_year', 
    'ACTRECEIVEDPRIORFISCALYEAR': 'act_received_prior_fiscal_year',
    'SHORTTERMDEBTMRECENTFISCALYEAR': 'short_term_debt_recent_fiscal_year',
    'SHORTTERMDEBTPRIORFISCALYEAR': 'short_term_debt_prior_fiscal_year',
    'LONGTERMDEBTRECENTFISCALYEAR': 'long_term_debt_recent_fiscal_year',
    'LONGTERMDEBTPRIORFISCALYEAR': 'long_term_debt_prior_fiscal_year',
    'REVENUEMOSTRECENTFISCALYEAR': 'revenue_most_recent_fiscal_year',
    'REVENUEPRIORFISCALYEAR': 'revenue_prior_fiscal_year', 
    'COSTGOODSSOLDRECENTFISCALYEAR': 'cost_goods_sold_recent_fiscal_year',
    'COSTGOODSSOLDPRIORFISCALYEAR': 'cost_goods_sold_prior_fiscal_year',
    'TAXPAIDMOSTRECENTFISCALYEAR': 'tax_paid_most_recent_fiscal_year',
    'TAXPAIDPRIORFISCALYEAR': 'tax_paid_prior_fiscal_year',
    'NETINCOMEMOSTRECENTFISCALYEAR': 'net_income_most_recent_fiscal_year',
    'NETINCOMEPRIORFISCALYEAR': 'net_income_prior_fiscal_year'
}, inplace = True )

# change data type
disclosure_dataframe = disclosure_dataframe.astype({
    'accession_number': 'object',
    'compensation_amount_description': 'object',
    'financial_interest': 'object',
    'security_offered_type': 'object',
    'security_offered_other_desc': 'object',
    'no_of_security_offered': 'float',
    'price': 'float',
    'price_determination_method': 'object',
    'offering_amount': 'float',
    'oversubscription_accepted': 'object',
    'oversubscription_allocation_type': 'string',
    'desc_oversubscription': 'string',
    'maximum_offering_amount': 'string',
    'deadline_date': 'datetime64[ns]',
    'current_employees': 'object',
    'total_assets_most_recent_fiscal_year': 'float',
    'total_assets_prior_fiscal_year': 'float',
    'cash_equity_most_recent_fiscal_year': 'float',
    'cash_equity_prior_fiscal_year': 'float',
    'act_received_recent_fiscal_year': 'float',
    'act_received_prior_fiscal_year': 'float',
    'short_term_debt_recent_fiscal_year': 'float',
    'short_term_debt_prior_fiscal_year': 'float',
    'long_term_debt_recent_fiscal_year': 'float',
    'long_term_debt_prior_fiscal_year': 'float',
    'revenue_most_recent_fiscal_year': 'float',
    'revenue_prior_fiscal_year': 'float', 
    'cost_goods_sold_recent_fiscal_year': 'float',
    'cost_goods_sold_prior_fiscal_year': 'float',
    'tax_paid_most_recent_fiscal_year': 'float',
    'tax_paid_prior_fiscal_year': 'float',
    'net_income_most_recent_fiscal_year': 'float',
    'net_income_prior_fiscal_year': 'float'
})

# combined_disclosure_df['accession_number'][1]

# combined_disclosure_df.info()
# combined_disclosure_df['current_employees'].notna()

disclosure_dataframe.info()
disclosure_dataframe.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23474 entries, 0 to 23473
Data columns (total 33 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   accession_number                      23474 non-null  object        
 1   compensation_amount_description       18670 non-null  object        
 2   financial_interest                    13979 non-null  object        
 3   security_offered_type                 18672 non-null  object        
 4   security_offered_other_desc           8181 non-null   object        
 5   no_of_security_offered                15020 non-null  float64       
 6   price                                 17692 non-null  float64       
 7   price_determination_method            10715 non-null  object        
 8   offering_amount                       18672 non-null  float64       
 9   oversubscription_accepted             18672 non-null  object        
 10

,accession_number,compensation_amount_description,financial_interest,security_offered_type,security_offered_other_desc,no_of_security_offered,price,price_determination_method,offering_amount,oversubscription_accepted,...,long_term_debt_recent_fiscal_year,long_term_debt_prior_fiscal_year,revenue_most_recent_fiscal_year,revenue_prior_fiscal_year,cost_goods_sold_recent_fiscal_year,cost_goods_sold_prior_fiscal_year,tax_paid_most_recent_fiscal_year,tax_paid_prior_fiscal_year,net_income_most_recent_fiscal_year,net_income_prior_fiscal_year
0,0001820003-20-000004,5.5% of the amount raised,NaN,Common Stock,NaN,10000.0,1.00,NaN,10000.00,Y,...,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,-55126.0,-262805.00
1,0001665160-20-001227,6.0 percent,Two percent (2%) of securities of the total am...,Common Stock,NaN,10869.0,0.92,NaN,9999.48,Y,...,1511247.57,1108913.1,69128.35,4596.2,63997.0,2321.95,800.0,0.0,-636984.0,-397503.57
2,0001826174-20-000002,commission of 7% on funds raised,Not applicable,Common Stock,NaN,10000.0,0.50,NaN,5000.00,Y,...,7250.00,0.0,76.00,0.0,16.0,0.00,0.0,0.0,60.0,0.00
3,0001669191-20-000558,Up to 4.9% of amount raised for a successful o...,None.,Common Stock,NaN,20000.0,0.50,At the issuer's discretion.,10000.00,Y,...,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00
4,0001669191-20-000556,Up to 4.9% of amount raised for a successful o...,None.,Common Stock,NaN,3216.0,3.11,At the issuer's discretion.,10001.76,Y,...,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,-183454.0,-61790.00


In [40]:
# COISSUER

coissuer_info_dataframe.rename(columns={
    'ACCESSION_NUMBER':'accession_number',
    'ID':'id',
    'ISEDGARFILER':'is_edgar_filer',
    'COISSUERCIK':'co_issuer_cik',
    'NAMEOFCOISSUER':'name_of_co_issuer',
    'LEGALSTATUSFORM':'legal_status_form',
    'LEGALSTATUSOTHERDESC':'legal_status_other_desc',
    'JURISDICTIONORGANIZATION':'jurisdiction_organization',
    'DATEINCORPORATION':'date_incorporation',
    'STREET1':'street_1',
    'STREET2':'street_2',
    'CITY':'city',
    'STATEORCOUNTRY':'state_or_country',
    'ZIPCODE':'zipcode',
    'COISSUERWEBSITE':'co_issuer_website'
},inplace=True)

coissuer_info_dataframe['date_incorporation'] = pd.to_datetime(coissuer_info_dataframe['date_incorporation'], format='%d-%b-%Y')

coissuer_info_dataframe.astype({
    'accession_number':'object',
    'id':'object',
    'date_incorporation':'datetime64[ns]'
})

# coissuer_info_dataframe
coissuer_info_dataframe.info()
coissuer_info_dataframe.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2122 entries, 0 to 2121
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   accession_number           2122 non-null   object        
 1   id                         2122 non-null   int64         
 2   is_edgar_filer             5 non-null      object        
 3   co_issuer_cik              7 non-null      float64       
 4   name_of_co_issuer          2122 non-null   object        
 5   legal_status_form          2122 non-null   object        
 6   legal_status_other_desc    3 non-null      object        
 7   jurisdiction_organization  2122 non-null   object        
 8   date_incorporation         2122 non-null   datetime64[ns]
 9   street_1                   2122 non-null   object        
 10  street_2                   2027 non-null   object        
 11  city                       2122 non-null   object        
 12  state_

,accession_number,id,is_edgar_filer,co_issuer_cik,name_of_co_issuer,legal_status_form,legal_status_other_desc,jurisdiction_organization,date_incorporation,street_1,street_2,city,state_or_country,zipcode,co_issuer_website
0,0001670254-21-000764,2,NaN,NaN,"BioSmart Research I EB, a series of Wefunder S...",Limited Liability Company,NaN,DE,2021-06-22,4104 24TH ST,PMB 8113,San Francisco,CA,94114,https://wefunder.com/
1,0001670254-21-000764,1,NaN,NaN,"BioSmart Research I, a series of Wefunder SPV,...",Limited Liability Company,NaN,DE,2021-06-22,4104 24TH ST,PMB 8113,San Francisco,CA,94114,https://wefunder.com/
2,0001670254-21-000762,2,NaN,NaN,"Invincible Entertainment Partners I EB, a seri...",Limited Liability Company,NaN,DE,2021-06-29,4104 24TH ST,PMB 8113,San Francisco,CA,94114,https://wefunder.com/
3,0001670254-21-000762,1,NaN,NaN,"Invincible Entertainment Partners I, a series ...",Limited Liability Company,NaN,DE,2021-06-29,4104 24TH ST,PMB 8113,San Francisco,CA,94114,https://wefunder.com/
4,0001670254-21-000760,1,NaN,NaN,"Nwicode I, a series of Wefunder SPV, LLC",Limited Liability Company,NaN,DE,2021-06-22,4104 24TH ST,PMB 8113,San Francisco,CA,94114,https://wefunder.com/


In [41]:
# ISSUER INFO

issuer_info_dataframe.rename(columns={
    'ACCESSION_NUMBER': 'accession_number',
    'ISAMENDMENT': 'is_amendment', 
    'PROGRESSUPDATE': 'progress_update',
    'NATUREOFAMENDMENT': 'nature_of_amendment',
    'NAMEOFISSUER': 'name_of_issuer',
    'LEGALSTATUSFORM': 'legal_status_form',
    'LEGALSTATUSOTHERDESC': 'legal_status_other_desc',
    'JURISDICTIONORGANIZATION': 'jurisdiction_organization',
    'DATEINCORPORATION': 'date_incorporation',
    'STREET1': 'street1',
    'STREET2': 'street2',
    'CITY': 'city',
    'STATEORCOUNTRY': 'state_or_country',
    'ZIPCODE': 'zipcode',
    'ISSUERWEBSITE': 'issuer_website',
    'COMPANYNAME': 'company_name',
    'COMMISSIONCIK': 'commission_cik',
    'COMMISSIONFILENUMBER': 'commission_file_number',
    'CRDNUMBER': 'crd_number',
},inplace=True)

issuer_info_dataframe['date_incorporation'] = pd.to_datetime(issuer_info_dataframe['date_incorporation'])

issuer_info_dataframe.astype({
    'date_incorporation':'datetime64[ns]'
})

issuer_info_dataframe.info()
issuer_info_dataframe.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23474 entries, 0 to 23473
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   accession_number           23474 non-null  object        
 1   is_amendment               23474 non-null  int64         
 2   progress_update            3575 non-null   object        
 3   nature_of_amendment        7862 non-null   object        
 4   name_of_issuer             23472 non-null  object        
 5   legal_status_form          22587 non-null  object        
 6   legal_status_other_desc    262 non-null    object        
 7   jurisdiction_organization  22587 non-null  object        
 8   date_incorporation         22587 non-null  datetime64[ns]
 9   street1                    22586 non-null  object        
 10  street2                    7114 non-null   object        
 11  city                       22587 non-null  object        
 12  stat

,accession_number,is_amendment,progress_update,nature_of_amendment,name_of_issuer,legal_status_form,legal_status_other_desc,jurisdiction_organization,date_incorporation,street1,street2,city,state_or_country,zipcode,issuer_website,company_name,commission_cik,commission_file_number,crd_number,ISCOISSUER
0,0001820003-20-000004,0,NaN,Updated physical address of company.,"Airo Capital Management, Inc.",Corporation,NaN,MD,2018-08-29,PO Box 340,NaN,Olney,MD,20830,https://www.airocapitalmanagement.com/,www.fundme.com,1683485.0,007-00078,2344863.0,NaN
1,0001665160-20-001227,0,NaN,NaN,The Luving Company,Corporation,NaN,DE,2016-05-12,PO BOX 9147,NaN,Marina Del Rey,CA,90295,www.kidsluv.com,"StartEngine Capital, LLC",1665160.0,007-00007,NaN,NaN
2,0001826174-20-000002,0,NaN,NaN,"MEVERO, INC.",Corporation,NaN,DE,2019-04-26,3260 HILLVIEW AVE,NaN,PALO ALTO,CA,94304,www.mevero.com,crowdsourcefunded.com,1667727.0,007-00027,283378.0,NaN
3,0001669191-20-000558,0,NaN,the issuer made minor image updates on the off...,"Forever Young Technologies, INC",Corporation,NaN,DE,2020-05-12,1042 N. Mountain Ave,Suite B 374,Upland,CA,91786,www.melanence.com,NetCapital Funding Portal Inc.,1669191.0,007-00035,283596.0,NaN
4,0001669191-20-000556,0,NaN,"minor disclosure added to the ""terms"" section ...",GenesisAI Corporation,Corporation,NaN,DE,2018-07-03,1010 Massachusetts Ave,NaN,CAMBRIDGE,MA,02138,https://www.genesisai.io/,NetCapital Funding Portal Inc.,1669191.0,007-00035,283596.0,NaN


In [42]:
submission_dataframes.info()
submission_dataframes.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23474 entries, 0 to 23473
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ACCESSION_NUMBER  23474 non-null  object 
 1   SUBMISSION_TYPE   23474 non-null  object 
 2   FILING_DATE       23474 non-null  int64  
 3   CIK               23472 non-null  float64
 4   FILE_NUMBER       23474 non-null  object 
 5   PERIOD            3284 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 1.1+ MB


,ACCESSION_NUMBER,SUBMISSION_TYPE,FILING_DATE,CIK,FILE_NUMBER,PERIOD
0,0001820003-20-000004,C/A,20200930,1820003.0,020-26982,NaN
1,0001665160-20-001227,C,20200930,1823056.0,020-26995,NaN
2,0001826174-20-000002,C,20200930,1826174.0,020-26994,NaN
3,0001669191-20-000558,C/A,20200930,1822208.0,020-26833,NaN
4,0001669191-20-000556,C/A,20200930,1797609.0,020-26652,NaN
